In [1]:
import pandas as pd
import gc
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

In [2]:
user_app_usage_lines = 651007719
pieces = 10
interval = int(user_app_usage_lines / pieces)
uau_skiprows = [range(0, i*interval) for i in range(pieces)]
user_app_usage_path = '/Volumes/nvsd/user_app_usage.csv'
user_app_usage = None

In [3]:
with open(user_app_usage_path, 'r') as origin:
    for i in tqdm(range(pieces)):
        n_interval = interval if i != 9 else user_app_usage_lines - interval * 9
        with open('/Volumes/miguch/data/user_app_usage%d.csv'%i, 'w') as splitted:
            for k in tqdm(range(n_interval), leave=False):
                line = origin.readline()
                splitted.write(line)